TensorFlow 的 Eager Execution 是一种命令式编程环境，可立即评估操作，无需构建图：操作会返回具体的值，而不是构建以后再运行的计算图。这样能让您轻松地开始使用 TensorFlow 和调试模型，并且还减少了样板代码。

In [1]:
# 此句主要是让python2.7的代码也能在python3上面运行
from __future__ import absolute_import, division, print_function

import tensorflow as tf

# 开启Eager Execution
tf.enable_eager_execution()

x = [[2.0]]
m = tf.matmul(x, x)
print("hello,{}".format(m))

hello,[[4.]]


In [2]:
a = tf.constant([[1, 2],
                 [3, 4]])
print(a)

tf.Tensor(
[[1 2]
 [3 4]], shape=(2, 2), dtype=int32)


In [3]:
b = tf.add(a, 1)
print(b)

tf.Tensor(
[[2 3]
 [4 5]], shape=(2, 2), dtype=int32)


In [4]:
print(a * b)

tf.Tensor(
[[ 2  6]
 [12 20]], shape=(2, 2), dtype=int32)


In [5]:
import numpy as np

c = np.multiply(a, b)
print(c)


[[ 2  6]
 [12 20]]


tf.Tensor.numpy 方法返回对象的值作为 NumPy ndarray

In [6]:
print(a.numpy())

[[1 2]
 [3 4]]


In [7]:
def fizzbuzz(max_num):
    counter = tf.constant(0)
    for num in range(max_num):
        num = tf.constant(num)
        if int(num % 3) == 0 and int(num % 5) == 0:
            print("FizeeBuzee")
        elif int(num % 3) == 0:
            print("Fizee")
        elif int(num % 5) == 0:
            print("Buzz")
        else:
            print(num)

        counter += 1

    return counter

fizzbuzz(10)


FizeeBuzee
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
Fizee
tf.Tensor(4, shape=(), dtype=int32)
Buzz
Fizee
tf.Tensor(7, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
Fizee


<tf.Tensor: id=98, shape=(), dtype=int32, numpy=10>

In [8]:
#tf.keras.Dense 表示全连接层
#tf.keras.qusuential 表示由现行堆叠的模型
# 以下两种生成模型的方法效果相同

model = tf.keras.Sequential([
    tf.keras.layers.Dense(10, input_shape=(784,)),
    tf.keras.layers.Dense(10)
])


class MNISTModel(tf.keras.Model):
    def __init__(self):
        super(MNISTModel, self).__init__()
        self.Dense1 = tf.keras.layers.Dense(units=10)
        self.Dense2 = tf.keras.layers.Dense(units=10)

    def call(self, input):
        result = self.Dense1(input)
        result = self.Dense2(input)
        result = self.Dense2(input)
        return result


model = MNISTModel()

In [10]:
import tensorflow.contrib.eager as tfe

w = tfe.Variable([[1.0]])
# tf.GradientTape()跟踪操作，可以计算梯度
with tf.GradientTape()as tape:
    loss = w * w
grad = tape.gradient(loss, [w])
print(grad)


[<tf.Tensor: id=181, shape=(1, 1), dtype=float32, numpy=array([[2.]], dtype=float32)>]


In [11]:
NUM_EXAMPLES = 1000
training_inputs = tf.random_normal([NUM_EXAMPLES])
noise = tf.random_normal([NUM_EXAMPLES])
training_outputs = training_inputs * 3 + 2 + noise

def prediction(input, weight, bias):
    return input * weight + bias

def loss(weights, biases):
    error = prediction(training_inputs, weights, biases) - training_outputs
    return tf.reduce_mean(tf.square(error))

def grad(weights, biases):
    with tf.GradientTape() as tape:
        loss_value = loss(weights, biases)
    return tape.gradient(loss_value, [weights, biases])


train_steps = 200
learning_rate = 0.01

W = tfe.Variable(5.)
B = tfe.Variable(10.)

print("Initial loss: {:.3f}".format(loss(W, B)))
for i in range(train_steps):
    dW, dB = grad(W, B)
    W.assign_sub(dW * learning_rate)
    B.assign_sub(dB * learning_rate)
    if i % 20 == 0:
        print("Loss as step :{:03d}:{:.3f}".format(i, loss(W, B)))

print("W = {},B = {}".format(W.numpy(), B.numpy()))


Initial loss: 67.669
Loss as step :000:65.079
Loss as step :020:30.041
Loss as step :040:14.190
Loss as step :060:7.008
Loss as step :080:3.750
Loss as step :100:2.270
Loss as step :120:1.597
Loss as step :140:1.290
Loss as step :160:1.150
Loss as step :180:1.086
W = 3.0622386932373047,B = 2.146221399307251


tfe.Variable 对象存储在训练期间访问的可变 tf.Tensor 值，以更加轻松地实现自动微分。模型的参数可以作为变量封装在类中

In [12]:
class Model(tf.keras.Model):
    def __init__(self):
        super(Model, self).__init__()
        self.W = tfe.Variable(5., name='weight')
        self.B = tfe.Variable(10., name='bias')
    def predict(self, inputs):
        return inputs * self.W + self.B


# 输入 输出数据
NUM_EXAMPLES = 2000
training_inputs = tf.random_normal([NUM_EXAMPLES])
noise = tf.random_normal([NUM_EXAMPLES])
training_outputs = training_inputs * 3 + 2 + noise

# 损失函数
def loss(model, inputs, targets):
    error = model.predict(inputs) - targets
    return tf.reduce_mean(tf.square(error))

# 梯度函数
def grad(model, inputs, targets):
    with tf.GradientTape() as tape:
        loss_value = loss(model, inputs, targets)
    return tape.gradient(loss_value, [model.W, model.B])

model = Model()
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)

print("Initial loss: {:.3f}".format(loss(model, training_inputs, training_outputs)))

#迭代训练
for i in range(300):
    grads = grad(model, training_inputs, training_outputs)
    # zip函数接受任意多个序列作为参数，将所有序列按相同的索引组合成一个元素是各个序列合并成的tuple的新序列，新的序列的长度以参数中最短的序列为准
    optimizer.apply_gradients(zip(grads, [model.W, model.B]),
                              global_step=tf.train.get_or_create_global_step())
    if i % 20 == 0:
        print("Loss at step {:03d}: {:.3f}".format(i, loss(model, training_inputs, training_outputs)))

print("Final loss: {:.3f}".format(loss(model, training_inputs, training_outputs)))
print("W = {}, B = {}".format(model.W.numpy(), model.B.numpy()))


Initial loss: 69.036
Loss at step 000: 66.346
Loss at step 020: 30.158
Loss at step 040: 14.004
Loss at step 060: 6.792
Loss at step 080: 3.573
Loss at step 100: 2.135
Loss at step 120: 1.493
Loss at step 140: 1.207
Loss at step 160: 1.079
Loss at step 180: 1.021
Loss at step 200: 0.996


Loss at step 220: 0.984
Loss at step 240: 0.979
Loss at step 260: 0.977
Loss at step 280: 0.976
Final loss: 0.976
W = 3.0077202320098877, B = 2.0057666301727295


In [13]:
def square(x):
    return tf.multiply(x, x)

grad = tfe.gradients_function(square)

print(square(3.))
print(grad(3.))

# 二次求导
gradgrad = tfe.gradients_function(lambda x: grad(x)[0])
print(gradgrad(3.))

# 三次求导
gradgradgrad = tfe.gradients_function(lambda x: gradgrad(x)[0])
print(gradgradgrad(3.))

def abs(x):
    return x if x > 0. else -x

grad = tfe.gradients_function(abs)

print(grad(3.))
print(grad(-3.))


tf.Tensor(9.0, shape=(), dtype=float32)
[<tf.Tensor: id=17922, shape=(), dtype=float32, numpy=6.0>]
[<tf.Tensor: id=17934, shape=(), dtype=float32, numpy=2.0>]
[None]
[<tf.Tensor: id=159, shape=(), dtype=float32, numpy=1.0>]
[<tf.Tensor: id=17959, shape=(), dtype=float32, numpy=-1.0>]


自定义梯度

In [14]:
@tf.custom_gradient
def log1pexp(x):
    e = tf.exp(x)
    # 自定义梯度的函数
    def grad(dy):
        return dy * (1 - 1 / (1 + e))
    return tf.log(1 + e), grad

grad_log1pexp = tfe.gradients_function(log1pexp)

print(grad_log1pexp(0.))

print(grad_log1pexp(100.))

[<tf.Tensor: id=17970, shape=(), dtype=float32, numpy=0.5>]
[<tf.Tensor: id=17981, shape=(), dtype=float32, numpy=1.0>]


虽然 Eager Execution 增强了开发和调试的交互性，但 TensorFlow Graph Execution 在分布式训练、性能优化和生产部署方面具有优势。不过，编写图形代码不同于编写常规 Python 代码，并且更难以调试